In [ ]:
# Extrair
import pandas as pd
URL = "https://balanca.economia.gov.br/balanca/bd/comexstat-bd/mun/EXP_2025_MUN.csv"
df = pd.read_csv(URL, sep=';')



In [ ]:
# Explorar
df.info()
df.sample(5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784166 entries, 0 to 784165
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   CO_ANO      784166 non-null  int64 
 1   CO_MES      784166 non-null  int64 
 2   SH4         784166 non-null  int64 
 3   CO_PAIS     784166 non-null  int64 
 4   SG_UF_MUN   784166 non-null  object
 5   CO_MUN      784166 non-null  int64 
 6   KG_LIQUIDO  784166 non-null  int64 
 7   VL_FOB      784166 non-null  int64 
dtypes: int64(7), object(1)
memory usage: 47.9+ MB


,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB
429867,2025,4,9018,603,SP,3448906,2,8003
337698,2025,8,4016,97,SP,3409205,256,14021
65747,2025,8,4016,169,RS,4318705,71,2232
107886,2025,6,9031,63,SC,4209102,98,10828
128059,2025,2,3921,63,SP,3454508,41025,154714


In [ ]:
# Transformar - Limpeza e Ajustes

# Criar coluna de data
df['DATA'] = pd.to_datetime(df['CO_ANO'].astype(str) + '-' + df['CO_MES'].astype(str) + '-01')

# Selecionar apenas colunas relevantes
df_simplificado = df[['DATA', 'SG_UF_MUN', 'SH4', 'VL_FOB', 'KG_LIQUIDO']].copy()

# Melhorar nomes de colunas
df_simplificado = df_simplificado.rename(columns={
    'SG_UF_MUN': 'UF',
    'SH4': 'Produto',
    'VL_FOB': 'Valor_USD',
    'KG_LIQUIDO': 'Peso_kg'
})

# Tratar nulos
df_simplificado = df_simplificado.dropna(subset=['Valor_USD', 'Peso_kg'])


In [ ]:
# Transformar - Agregação

# Total por Estado e Mês
resumo_uf = (
    df_simplificado
    .groupby(['UF', 'DATA'])[['Valor_USD', 'Peso_kg']]
    .sum()
    .reset_index()
)

# Preço médio por quilo
resumo_uf.loc[resumo_uf['Peso_kg'] == 0, 'Peso_kg'] = pd.NA  # evita divisão por zero
resumo_uf['Preco_medio_USD_kg'] = resumo_uf['Valor_USD'] / resumo_uf['Peso_kg']

# Resultado
resumo_uf.head()

,UF,DATA,Valor_USD,Peso_kg,Preco_medio_USD_kg
0,AC,2025-01-01,4933169,3237472.0,1.523772
1,AC,2025-02-01,4825068,2399102.0,2.011198
2,AC,2025-03-01,5622040,2533483.0,2.219095
3,AC,2025-04-01,7781146,3112825.0,2.499706
4,AC,2025-05-01,4799819,1775893.0,2.702764


In [ ]:
# Carregar
resumo_uf.to_csv("exportacoes_resumo.csv", index=False)

from google.colab import files

#Download para a sua máquina local
files.download("exportacoes_resumo.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>